In [ ]:
#Nem todas as palavras das stopwords estão sendo filtradas corretamente

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#novas stop words

def addStopWords():
    newStopWords = ['ColateraisQuais', 'males', 'pode', 'causar', 'Alguns', 
                    'efeitos', 'colaterais', 'comuns', 'Amitriptilina', 
                    'incluem', 'Fluoxetina', 'Sertralina\u200b', 'podem', 'incluir',
                    'Bupropiona', 'reações', 'Os', 'efeitos', 'adversos', 'comuns', 
                    'provocados', 'uso', 'paroxetina', 'são', 'abundante', 'frequentes', 
                    'problemas', '(', '1', '1%', 'Trazodona', 'Pacientes', 'sexo', 'masculino',
                    'tratamento', 'devem', 'interromper', 'imediato', 'administração', 
                    'medicamento', 'consultar', 'médico', 'dá', 'pode', 'provocar', 'Espran',
                    'atingir', "causar", "mulheres", 'ocorrer', 'operar', 'máquinas', 'certeza', 
                    'medicamento', 'vez', 'fluoxetina', 'interferir', 'capacidade', 'ação', 
                    'pacientes', 'tratados', 'produto', 'relatados', 'seguintes', 'fluoxetina', 
                    'todoSintomas', 'autonômicos', 'incluindo', 'caracterizada', 'conjunto', 
                    'características', 'clínicas', 'atividade', 'autônomo', 'camadas', 'profundas', 
                    'incluindo', 'sintomas', 'associados', 'frequentemente', 'observadas', 
                    'sintomas', 'geralmente', 'leves', 'moderados', 'limitados', 'entanto', 
                    'alguns', 'pacientes', 'ser', 'graves', 'prolongados', 'razão', 'é', 
                    'aconselhável', 'cloridrato', 'necessário', 'dose', 'deve', 'ser', 
                    'gradualmente', 'reduzida', 'relatos', 'ocorridos', 'sistemas', 
                    'reação', 'semelhante', 'doença', 'soro', 'ingestão', 'oral', '150', 
                    'mg', 'Trazodona', 'concentração', 'máxima', 'organismo', 'alcançada', 
                    'tempo', 'máximo', '4', 'horas', 'após', 'ingestão', '1', '%', 'ADVERSOSOs'
                    'apresentarem', 'inapropriado']
    for w in newStopWords:
        if w not in stopWords:
            stopWords.add(w)

In [ ]:
#import das bulas por url

Fluoxetina = urllib.request.urlopen('https://www.bulario.com/fluoxetina/').read()
Amitriptilina = urllib.request.urlopen('https://www.bulario.com/amitriptilina/').read()
Sertralina = urllib.request.urlopen('https://www.bulario.com/sertralina_50_mg/').read()
Bupropiona = urllib.request.urlopen('https://www.bulario.com/bupropiona/').read()
Paroxetina = urllib.request.urlopen('https://www.bulario.com/paroxetina/').read()
Citalopram = urllib.request.urlopen('https://www.bulario.com/citalopram/').read()
Imipramina = urllib.request.urlopen('http://www.medicamentosesaude.com/bula-de-remedio-imipramina/').read()
Selegilina = urllib.request.urlopen('http://www.medicamentosesaude.com/bula-de-remedio-selegilina/').read()
Trazodona = urllib.request.urlopen('https://www.bulario.com/trazodona/').read()
Escitalopram = urllib.request.urlopen('https://www.bulario.com/espran/').read()

In [ ]:
#StopWords and punctuations
stopWords = []
stopWords = set(stopwords.words('portuguese'))
addStopWords()
#print(stopWords)
punctuations = list(string.punctuation)
#print(punctuations)

In [ ]:
'''
Definição dos filtros das palavras. Há 4 diferentes alterando o range do text.find
de acordo com as diferentes bulas que não são padronizadas. Há a função poncIndex()
que fica buscando as pontuações no meio das palavras recursivamente. A ideia era ter 
implementado juntamente dos filtros, porém não dava certo, mas como função auxilar, 
deu certo.

Havia muita palavra com pontuação junta, então criamos os 3 casos abaixo.

Após tem o if gigantesco que foi necessário, pois as palavras que tinham pontuação
estavam entrando repetidamente na lista.
'''
def filtro (bytes):
    
    soup = BeautifulSoup(bytes, "lxml")
    text = soup.get_text(strip=True) 
    text = text[text.find('Colaterais'):text.find('Contraindicações')]
    text = [t for t in text.split()]
    
    #print ("Palavras:")
    for t in text:
        if t not in stopWords and t not in punctuations:
            if any(c in t for c in punctuations):
                for w in t: 
                    if w in punctuations:
                        temp = []
                        #caso 1: amora)
                        if t.index(w) == len(t)-1:
                            temp = t[:t.index(w)]
                            if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                wordsFiltered.append(temp)
                                break
                            else: break
                        #caso 2: amora(casa
                        elif t.index(w) != 0:
                            temp = t[:t.index(w)]
                            #print(temp)
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                        #caso 3: (amora
                        elif t.index(w) == 0:
                            temp = t[1:]
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                                
            else:
                wordsFiltered.append(t)
    #print(wordsFiltered)
    
    return 

In [ ]:
def filtro2 (bytes):
    
    soup = BeautifulSoup(bytes, "lxml")
    text = soup.get_text(strip=True) 
    text = text[text.find('Colaterais'):text.find('consultar')]
    text = [t for t in text.split()]
    
    #print ("Palavras:")
    for t in text:
        if t not in stopWords and t not in punctuations:
            if any(c in t for c in punctuations):
                for w in t: 
                    if w in punctuations:
                        temp = []
                        #caso 1: amora)
                        if t.index(w) == len(t)-1:
                            temp = t[:t.index(w)]
                            if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                wordsFiltered.append(temp)
                                break
                            else: break
                        #caso 2: amora(casa
                        elif t.index(w) != 0:
                            temp = t[:t.index(w)]
                            #print(temp)
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                        #caso 3: (amora
                        elif t.index(w) == 0:
                            temp = t[1:]
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                             
            else:
                wordsFiltered.append(t)
    #print(wordsFiltered)
    
    return 

In [ ]:
def filtro3 (bytes):
    
    soup = BeautifulSoup(bytes, "lxml")
    text = soup.get_text(strip=True) 
    text = text[text.find('ADVERSOS'):text.find('RECEITUÁRIO')]
    text = [t for t in text.split()]
    
    #print ("Palavras:")
    for t in text:
        if t not in stopWords and t not in punctuations:
            if any(c in t for c in punctuations):
                for w in t: 
                    if w in punctuations:
                        temp = []
                        #caso 1: amora)
                        if t.index(w) == len(t)-1:
                            temp = t[:t.index(w)]
                            if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                wordsFiltered.append(temp)
                                break
                            else: break
                        #caso 2: amora(casa
                        elif t.index(w) != 0:
                            temp = t[:t.index(w)]
                            #print(temp)
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                        #caso 3: (amora
                        elif t.index(w) == 0:
                            temp = t[1:]
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                             
            else:
                wordsFiltered.append(t)
    #print(wordsFiltered)
    
    return 

In [ ]:
def filtro4 (bytes):
    
    soup = BeautifulSoup(bytes, "lxml")
    text = soup.get_text(strip=True) 
    text = text[text.find('Colaterais'):text.find('Advertências')]
    text = [t for t in text.split()]
    
    #print ("Palavras:")
    for t in text:
        if t not in stopWords and t not in punctuations:
            if any(c in t for c in punctuations):
                for w in t: 
                    if w in punctuations:
                        temp = []
                        #caso 1: amora)
                        if t.index(w) == len(t)-1:
                            temp = t[:t.index(w)]
                            if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                wordsFiltered.append(temp)
                                break
                            else: break
                        #caso 2: amora(casa
                        elif t.index(w) != 0:
                            temp = t[:t.index(w)]
                            #print(temp)
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                        #caso 3: (amora
                        elif t.index(w) == 0:
                            temp = t[1:]
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                             
            else:
                wordsFiltered.append(t)
    #print(wordsFiltered)
    
    return 

In [ ]:
def  poncIndex(str):
    if str not in stopWords and str not in punctuations:
        if any(c in str for c in punctuations):
            for w in str: 
                if w in punctuations:
                        temp = []
                        #caso 1: amora)
                        if str.index(w) == len(str)-1:
                            temp = str[:str.index(w)]
                            if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                wordsFiltered.append(temp)
                                break
                            else: break
                        #caso 2: amora(casa
                        elif str.index(w) != 0:
                            temp = str[:str.index(w)]
                            #print(temp)
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                        #caso 3: (amora
                        elif str.index(w) == 0:
                            temp = str[1:]
                            if temp.isalpha():
                                if (temp not in stopWords) and (len(wordsFiltered)>0) and (temp not in wordsFiltered[-1]) and (wordsFiltered[-1] not in temp):
                                    wordsFiltered.append(temp)
                                    break
                                else: break
                            else:
                                poncIndex(temp)
                             
        else:
            wordsFiltered.append(str)
    return

In [ ]:
#Filtragem das stopWords

wordsFiltered = []
fluoxetina = []
filtro(Fluoxetina)
fluoxetina.extend(wordsFiltered) 

In [ ]:
wordsFiltered = []
amitriptilina = []
filtro(Amitriptilina)
amitriptilina = wordsFiltered

In [ ]:
wordsFiltered = []
sertralina = []
filtro(Sertralina)
sertralina = wordsFiltered

In [ ]:
wordsFiltered = []
bupropiona = []
filtro(Bupropiona)
bupropiona = wordsFiltered

In [ ]:
wordsFiltered = []
paroxetina = []
filtro(Paroxetina)
paroxetina = wordsFiltered

In [ ]:
wordsFiltered = []
citalopram = []
filtro(Citalopram)
citalopram = wordsFiltered

In [ ]:
wordsFiltered = []
imipramina = []
filtro3(Imipramina)
imipramina = wordsFiltered

In [ ]:
wordsFiltered = []
selegilina = []
filtro3(Selegilina)
selegilina = wordsFiltered

In [ ]:
wordsFiltered = []
trazodona = []
filtro2(Trazodona)
trazodona = wordsFiltered

In [ ]:
wordsFiltered = []
escitalopram = []
filtro4(Escitalopram)
escitalopram = wordsFiltered

In [ ]:
#Função que une os sintomas não repetindo-os
def juntaSintomas(list):
    for w in list:
        if w not in sintomas:
            sintomas.append(w)

In [ ]:
sintomas = []

juntaSintomas(fluoxetina)
juntaSintomas(amitriptilina)
juntaSintomas(sertralina)
juntaSintomas(bupropiona)
juntaSintomas(paroxetina)
juntaSintomas(citalopram)
juntaSintomas(imipramina)
juntaSintomas(selegilina)
juntaSintomas(trazodona)
juntaSintomas(escitalopram)

In [ ]:
#Cria matriz que conta as repetições de cada sintoma por medicamento
cnt = np.zeros((11, 253))
i = 0

for w in sintomas:
    
    cnt[0][i] = fluoxetina.count(w)
    cnt[1][i] = amitriptilina.count(w)
    cnt[2][i] = sertralina.count(w)
    cnt[3][i] = bupropiona.count(w)
    cnt[4][i] = paroxetina.count(w)
    cnt[5][i] = citalopram.count(w)
    cnt[6][i] = imipramina.count(w)
    cnt[7][i] = selegilina.count(w)
    cnt[8][i] = trazodona.count(w)
    cnt[9][i] = escitalopram.count(w)
    cnt[10][i] = cnt[9][i] + cnt[8][i] + cnt[7][i] + cnt[6][i] + cnt[5][i] + cnt[4][i] + cnt[3][i] + cnt[2][i] + cnt[1][i] + cnt[0][i] 
    
    i = i + 1

In [ ]:
#Array com o nome dos sintomas
remedio = ['fluoxetina',
'amitriptilina',
'sertralina',
'bupropiona',
'paroxetina',
'citalopram',
'imipramina',
'selegilina',
'trazodona',
'escitalopram',
'total']

In [ ]:
#Criação do dataFrame
data = pd.DataFrame(data = cnt,
                   columns = sintomas,
                   index = remedio)
data

,falta,apetite,náusea,diarreia,dirigir,veículos,desempenho,afetado,julgamento,pensamento,secura,boca,sudorese,aumento,diâmetro,vasos,sanguíneos,vasodilatação,calafrios,hipersensibilidade,coceira,prurido,erupções,pele,urticária,alérgica,grave,anafilactoide,inflamação,vasculite,seguida,inchaço,angioedema,síndrome,serotonérgica,alterações,estado,mental,neuromuscular,combinação,...,rash,enzimas,hepáticas,icterícia,secreção,antidiurético,ganho,retenção,agranulocitose,virada,noradrenérgica,ADVERSOSOs,hipotensão,ortostática,Menos,comumente,angina,edema,periférico,síncope,depressão,noctúria,hiperplasia,prostática,coreia,discinesias,borrada,desagradável,apresentarem,nariz,entupido,coriza,dificuldades,bocejar,sensação,picadas,dores,musculares,articulações,sexuais
fluoxetina,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,3.0,1.0,2.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
amitriptilina,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sertralina,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bupropiona,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
paroxetina,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
citalopram,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
imipramina,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
selegilina,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trazodona,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
escitalopram,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
#Exportação do dataFrame para melhor visualização
data.to_csv('deboraMatheus.csv')

In [ ]:
#Identifação das ocorrências
print("Termos de maior ocorrência:")
print ("Total: %s" %sintomas[np.argmax(cnt[10])])
print ("Fluoxetina: %s" %sintomas[np.argmax(cnt[0])])
print ("Amitriptilina: %s" %sintomas[np.argmax(cnt[1])])
print ("Sertralina: %s" %sintomas[np.argmax(cnt[2])])
print ("Bupropiona: %s" %sintomas[np.argmax(cnt[3])])
print ("Paroxetina: %s" %sintomas[np.argmax(cnt[4])])
print ("Citalopram: %s" %sintomas[np.argmax(cnt[5])])
print ("Imipramina: %s" %sintomas[np.argmax(cnt[6])])
print ("Selegilina: %s" %sintomas[np.argmax(cnt[7])])
print ("Trazodona: %s" %sintomas[np.argmax(cnt[8])])
print ("Escitalopram: %s" %sintomas[np.argmax(cnt[9])])


Termos de maior ocorrência:
Total: boca
Fluoxetina: pele
Amitriptilina: apetite
Sertralina: diarreia
Bupropiona: coceira
Paroxetina: apetite
Citalopram: apetite
Imipramina: síndrome
Selegilina: diarreia
Trazodona: boca
Escitalopram: aumento
